# Stat 157 Final Writeup: Permute Package Contributions


## Our Project
Throughout this semester, we set out to implement extended functionality to the Permute package developed here at UC Berkeley. Our contributions this semester are the following:
* One Sample Confidence Tests for Symmetric Distributions
* One Sample Confidence Tests for Population Percentiles.
* Confidence Tests for Regression
* Hoeffding's Inequality
* Estimation of Means of Bounded Distributions

### Pull Request Link:
https://github.com/statlab/permute/pull/134

## One Sample Confidence Tests for Symmetric Distributions

### Background
With only assumptions of an i.i.d. sample $x_t$, how can we check whether our sample's center is the true center of the population given that our is symmetric about a given center $c$? 

Solving this question gives yet another way to measure the center of a distribution, but with no assumptions about the distribution, other than that it is symmetric over a certain value.

The main approach is to use the assumption that $c$ is the center of the symmetric distribution as the null hypothesis and create a large enough number of new samples that have datapoints randomly rotated/orbitted around $c$. For each of these samples, a test statistic of the center will be taken from the sample and is compared with the observed test statistic of the original sample.

The Permute package already has a method to go about this (one_sample in core.py), but only set for $c=0$. What we had to do is make it applicable for any given center $c$.


### What we did
When creating new samples $\hat X_n$ that orbit $c$, For each datapoint $\hat X_i$ in that new sample, the value of the point would be $\hat X_i = c + (X_i - c) \cdot (1 - 2 B(\frac{1}{2}))$. Where $B(\frac{1}{2})$ is a Bernoulli random variable that is equal to 1 with chance $\frac{1}{2}$ and 0 otherwise. For each of these new samples, the test statistic for center would be applied to it, in turn creating an empirical distribution of the center statistic on samples of size n. Depending on the alternative of the test, it will be checked whether the center statistic of each of these new samples is either greater or less than the observed center statistic of the initial sample. For an alternative of 'greater', the number of newly created samples with a center statistic greater than the observed statistics, are counted in `hits` and the p-value would be the `hits` divided by `reps`, which is the number of new samples that were generated through the above equation.

The inputted data is a `numpy` array in Python, so the above equation was done with parallel vector computation.

### Code Snippet

In [7]:
# Alternative is 'greater'
def one_sample(x, center = 0, tst_fun = np.mean, reps=10**5):
    tst = tst_fun(stat)
    hits = np.sum(
        [tst_fun(center + (x - center) * (1 - 2 * prng.binomial(1, .5, size=n))) >= tst
            for i in range(reps)])
    return (hits / reps), tst

### Unit Testing

The above code snippet is just the general logic of what was modified in the `one_sample` function. The actual `one_sample` function in the package has more arguments and control statements within the function that 

## One Sample Confidence Tests for Percentiles

### Background
With only assumptions of an IID sample $X_n$, how can we predict the chance that a value $q_p$ is the $p$th percentile of the population?

Unlike the last problem, this one we had to come up with the methods and problems ourselves with the help of Professor Stark.

### What We Did
We learned from Professor Stark that this problem can be framed easily into a Binomial Confidence test. If $q_p$ is the $p$th percentile of the population, then for every value observed from the population, there is a $\frac{p}{100}$ chance that it would be at or below $q_p$ ($P(X <= q_p) = p$). That means that the distribution of the number of a datapoints in a random sample $X_n$ that are less than or equal to $q_p$ ($X_p$) is approximately $Binom(n, \frac{p}{100})$. We can therefore test if $\hat q_p$ is the $p$th percentile by getting a cdf of $Binom(n, p)$ at $\hat X_p$.

Formally, the test statistic $\hat X_p$ is the number of observations of the observed samples at or below $\hat q_p$.  The null hypothesis is that $\hat X_p = X_p$. If the alternative was that $\hat X_p < X_p$ (less), we would reject the null hypothesis if the chance that $X_p$ is less than $\hat X_p$ is lower than $\alpha$ ($P(X_p \le \hat X_p) \le \alpha $). This value can be obtained using a Binomial CDF ($BinomCDF(\hat X_p, n, \frac{p}{100})$). For an alternative of $\hat X_p > X_p$, $1 -BinomCDF(\hat X_p, n, \frac{p}{100})$ is used. 

The function to perform this test that we implemented in `permute.core`, named `one_sample_percentile`, takes in a sample `x`, a percentile value `p`, and a proposed `p`th percentile of the population, `x_p` (Equivalent to $\hat q_p$). This function would simply get the number of values less than or equal to `x_p` and make a call to a binomial cdf function. 

The binomial cdf function used is `scipy.stat.binom.cdf`. There is an existing binomial cdf function used in the package (`binomial_p`), but that wasn't chosen to be used at the end for reasons of efficiency and the strictly binomial closed form that this confidence test calls for.

Depending on the given alternative, what's done to the binomial cdf is different. The binomial cdf is kept the same for an alternative of $\hat X_p \le X_p$ (less), but for $\hat X_p \le X_p$ (greater), it would be 1 minus the binomial cdf. A code snippet for `less` as the alternative hypothesis is shown here:


### Code Snippet

In [ ]:
# Alternative is 'less'
def one_sample_percentile(x, x_p, p=50):
    num_under = np.sum(x <= x_p)
    return binom.cdf(num_under, n=len(x), p=p/100)
    

## Confidence Intervals for One Sample Tests

### Background
In the Permute package, there are already methods to make confidence intervals for any two-sample permutation tests, but none to make create confidence intervals for the above one sample percentile and test.

`scipy.optimize.brentq`, a root finding optimizer, is the main mechanism used to create the confidence intervals. A Python lambda function would be created that is equivalent to 0 when a desired confidence level is outputted from the confidence test.

An example of such a function is `lambda q: cl - conf_test(tune_param=q, alternative="less")`. Let's say that the confidence level `cl` was 0.95 and the confidence test `conf_test` was testing the probability that the null distribution is less than `q`. If the call to `conf_test` resulted in 0.95, the result of the function would be 0 and an upper bound `upp_bound` for the confidence interval will be found since it's known that 0.95 of the null distribution is less than `upp_bound`. This result can then be used to find a confidence interval where the alternative is greater.

In the same example for an alternative of 'less than', a lower bound `low_bound` could be found by calling `brentq` on the lambda function `lambda q: cl - conf_test(tune_param=q, alternative="greater")`. Notice that the only the `alternative` argument is changed in the call to `conf_test`.

For a two-sided confidence interval, `cl` is modified so that both an upper bound and a lower bound can be found through two calls to `brentq` that capture probability `cl` within the interval.

Also in `brentq`, a range has to be specified to do the root optimization over. In the 


### What We Did
The bulk of what we did for this task was to understand how the `scipy.optimize.brentq` could be used to find a confidence interval in the first place. 

**One sample confidence interval for center of a symmetric distribution:**
For the one sample test for the center of a symmetric distribution, we tuned `brentq` over the inputted `center` argument in `one_sample`. `center` is the hypothesized center of a sample `x` that is from an assumed symmetric null distribution around `center`. `one_sample` with an alternative of `greater` takes in a center statistic (ie. mean, median) and a sample, and finds the probability that the samples drawn from the null hypothesized distribution centered about `center` have a center statistic that is **greater** than the observed center statistic of the sample. The opposite is done for an alternative of `less`.

Because `center` is used to create a null hypothesis, the lower bound of the confidence interval would actually have to be the value of `center` where there is a `cl` chance that the center statistic of a sample drawn from the population is **less than** the observed center statistic of the inputted sample. Therefore the lambda function would be a `lambda q: cl - one_sample(x, alternative='less')` for the lower bound. For the upper bound the alternative will be `'greater'`.

`shift_limit`, which would represent the amount to add and subtract from a percieved center of the confidence interval was chosen to be the range of the data. This may not be the best shift limit to pick for samples with low variance since it may lead in brentq bounds that don't start/end with opposite signs. This is something that can be improved upon in the near future.

The code snippet is below:

In [ ]:
def one_sample_conf_int(x, cl=0.95, alternative="two-sided"):
    shift_limit = np.max(x) - np.min(x)
    tst = np.mean(x)
    ci_low = tst - shift_limit
    ci_upp = tst + shift_limit

    if alternative == 'two-sided':
        cl = 1 - (1 - cl) / 2

    if alternative != "upper":
        g = lambda q: cl - one_sample(x, alternative="less", seed=seed, \
            reps=reps, stat=stat, center=q)[0]
        ci_low = brentq(g, tst - 2 * shift_limit, tst + 2 * shift_limit)

    if alternative != "lower":
        g = lambda q: cl - one_sample(x, alternative="greater", seed=seed, \
            reps=reps, stat=stat, center=q)[0]
        ci_upp = brentq(g, tst - 2 * shift_limit, tst + 2 * shift_limit)

    return ci_low, ci_upp

**One sample confidence interval for percentile of a distribution:**
For this test, we tuned `brentq` over the inputted `x_p` argument in `one_sample_percentile`. The `x_p` creates the test statistic of the number of values in the sample that are under a proposed `p`th percentile, `num_under`. `one_sample_percentile` with an alternative of `less` would test the probability that for any random sample of size `n` from the population, the number of values at or under the true `p`th percentile is **less** than `num_under`. An alternative of `greater` would yield the chance of the number of values under the true `p`th percentile of any `n` sized random sample of the population to be **greater** than `num_under`.

For the percentile confidence interval, the inputted `x_p` argument is used to build up the alternative hypothesis of the test. Therefore, a lower bound is created by finding the value `x_p` such that `one_sample_percentile` with an alternative of `greater` at confidence level `cl`. On the other hand, an upper bound can be created by finding the value `x_p` such that `one_sample_percentile` with an alternative of `less` has a result of `cl`.

The range in which to run `brentq` simply starts at the minimum and maximum of the sample since the 

Talking to professor stark further, it was found that the confidence interval must consist of values that are actually present in the given sample. For this to be done, the `numpy.searchsorted` method was used to find the index of the sorted sample where if you insert a bound of the confidence interval before it, it would maintain the order of the array. For the lower bound, we got that index - 1 and for the upper bound we just used the given index to find conservative bounds. These were also tested through our unit tests.

The code snippet is below:

In [ ]:
def one_sample_percentile_ci(x, p=50, cl=0.95, alternative="two-sided", seed=None, reps=10**5):
    ci_low = np.min(x)
    ci_upp = np.max(x)
    
    x = np.sort(x)

    if alternative == 'two-sided':
        cl = 1 - (1 - cl) / 2

    if alternative != "upper":
        g = lambda q: cl - one_sample_percentile(x, q, p=p, alternative="greater")
        ci_low = brentq(g, ci_low, ci_upp)
        np.searchsorted(x, ci_low, side="left")

    if alternative != "lower":
        g = lambda q: cl - one_sample_percentile(x, q, p=p, alternative="less")
        ci_upp = brentq(g, ci_low, ci_upp)
        np.searchsorted(x, ci_upp, side="right")

## Permutation Tests for Simple Linear Regression

### Introduction: T-test for Regression Coefficient

Typically for linear regression, we have a data set with two variables $\{(x_i, y_i)\}_{i=1}^n$, and we fit the regression equation
$$y_i = a + \beta x_i + \epsilon_i$$

Where $\epsilon_i$ is the error term, and the regression coefficients $a$ and $\beta$ minimizes the sum of square differences: $\sum_{i=1}^n(y_i - a - \beta x_i)^2$.

Once we have these regression coefficients, you can test the significance of $\beta$ using a t-test. Using this method you test the null hypothesis: $H_0 : \beta = 0$. Then using the standard error of the coefficient we can calculate the test statistic

$$se(\beta) = \sqrt{\frac{\frac{1}{n-2}\sum_{i=1}^n\epsilon_i^2}{\sum_{i=1}^n(x_i - \bar{x})}}, \quad\quad t^* = \frac{\beta}{se(\beta)}$$

Using a t-distribution with $n-2$ degrees of freedom, we can find the desired p-value.

However, these tests can only be carried out if we assume that the error terms, $\epsilon_i$ are normally distributed with mean 0 and variance $\sigma^2$. If we do not have these conditions then we can not interpret the results of this test.

We introduce a different permutation-based method to test significance that has no requirement of the distribution of the data. Under the assumption of the null hypothesis $H_0:\beta = 0$, we see that our regression relationship becomes 
$$y_i = a + \epsilon_i$$

This indicates that there is no relationship between the $x_i$'s and the $y_i$'s. Therefore if we were to randomly shuffle the $\{x_i\}$ and $\{y_i\}$ pairs to obtain new pairs of data of the form $\{(x_{\pi_i}, y_i)\}$, where the $x_i$'s are now in some new permuted order. Now when we solve for the $\beta_{\pi}$ we expect that the magnitude of the coefficient to be similar to the value of our original $\beta$, which was solved for with the original data. We can generate a p-value by taking the fraction of occurrences where $|\beta_{\pi}| > |\beta|$ over a large number of trials of permuting the data. 


### Code Snippet

In [ ]:
def reg_coeff(x, y, reps, cl, seed):
    true_slope = abs(linregress(x, y)[0])
    prng = get_prng(seed)
    slopes = []
    for _ in range(reps):
        shuffle_x = prng.permutation(x)
        slope = linregress(shuffle_x, y)[0]
        slopes.append(slope)

    p_value = sum(abs(np.array(slopes)) > true_slope)/float(reps)

    return p_value

## Using Hoeffding's Inequality for Confidence Intervals

### Introduction

A common question to ask in statistics is the likeliness that a random variable $Z$ is close to its mean, $\mathbb{E}[Z]$. Most students have seen some inequalities that gives the probability bounds on certain random variables, such as Markov's inequality and Chebyshev's inequality. Hoeffding's inequality is an important probability bound on the sum of bounded random variables.

#### Hoeffding's Inequality
Let $Z_1, \dots, Z_n$ be independent bounded random variables with $Z_i \in [a_i,b_i]$ for all $i$, where $-\infty < a \leq b < \infty$. Then
$$\mathbb{P}\left(\frac{1}{n}\sum_{i=1}^n(Z_i - \mathbb{E}[Z_i]) \geq t\right) \leq exp\left(-\frac{2n^2t^2}{\sum_{i=1}^n(b_i-a_i)^2}\right)$$
and 
$$\mathbb{P}\left(\frac{1}{n}\sum_{i=1}^n(Z_i - \mathbb{E}[Z_i]) \leq -t\right) \leq exp\left(-\frac{2n^2t^2}{\sum_{i=1}^n(b_i-a_i)^2}\right)$$
for all $t \geq 0$.


### Derivation of Confidence Interval

Using this probability bound we can create a confidence interval for the mean of these independent observations. If we set 
$$t = \sqrt{\frac{-\ln{\alpha}\sum_{i=1}^n(b_i-a_i)^2}{2n^2}}$$
The probability that the observed mean is greater than the theoretical mean 
$$\mathbb{P}\left(\bar{Z} - \mu \geq \sqrt{\frac{-\ln{\alpha}\sum_{i=1}^n(b_i-a_i)^2}{2n^2}}\right) \leq exp\left( \frac{2n^2\ln{\alpha}\sum_{i=1}^n(b_i-a_i)^2}{2n^2\sum_{i=1}^n(b_i-a_i)^2}  \right) = e^{\ln(\alpha)} = \alpha$$
Thus the interval 
$$\boxed{\left[\bar{Z} - \sqrt{\frac{-\ln{(\alpha)}\sum_{i=1}^n(b_i-a_i)^2}{2n^2}}, \frac{1}{n}\sum_{i=1}^n b_i\right]}$$
is a one-sided confidence interval for the theoretical mean, with confidence level $1-\alpha$.\\\\
Through a similar derivation we have a two-sided confidence interval for the theoretical mean with confidence level $1-\alpha$ is given by
$$\boxed{\left[\bar{Z} - \sqrt{\frac{-\ln{(\alpha/2)}\sum_{i=1}^n(b_i-a_i)^2}{2n^2}}, \bar{Z} + \sqrt{\frac{-\ln{(\alpha/2)}\sum_{i=1}^n(b_i-a_i)^2}{2n^2}}\right]}$$

### Code Snippet

In [ ]:
def hoeffding_conf_int(x, N, lower_bound, upper_bound, cl = 0.95, alternative="one-sided"):
    tau_sq = np.sum((upper_bound - lower_bound)**2)
    x_bar = np.mean(x)

    if alternative == "one-sided":
        hCrit = np.sqrt(-math.log(1-cl)*tau_sq/(2*N**2))
        ci_low = x_bar - hCrit
        ci_upp = max_upper

    if alternative == "two-sided":
        hCrit = np.sqrt(-math.log((1-cl)/2)*tau_sq/(2*N**2))
        ci_low, ci_upp = x_bar - hCrit, x_bar + hCrit
    
    return ci_low, ci_upp

# KS Goodness of Fit Using Permute
We added a KS stats function in `two_sample` function and created a wrapper `two_sample_fit` to drive the goodness of fit test. We expect future contributions of other test stats to this function.

### Code Snippet

In [2]:
# This is the test stats function integrated into two_sample()
ks = lambda u, v: np.max([abs(sum(u <= a) / len(x) - sum(v <= a) / len(y))
                          for a in set(u).intersection(set(y))])

def two_sample_fit(x, y, alpha, method='KS'):
    if method != 'KS':
        raise ValueError('The function does not support ' + method + ' test.')

    _, stats = two_sample(x, y, stat=method)
    c = math.sqrt((len(x) + len(y))) / (len(x) * len(y))
    c *= math.sqrt(-0.5 * math.log(alpha * 0.5))
    return stats <= c

# SPRT
The model aims to determine the quality of a batch of products by minimal sampling.
The idea is to sample the batch sequentially until a decision can be made whether
the batch conforms to specification and can be accepted or that it should be rejected.

We consider a simple hypothesis $H_0: \theta = \theta_0$ against
a simple alternative $H_1: \theta = \theta_1$

$$\Lambda_n = \log \frac{L(\theta_1; X_1, ..., X_n)}{L(\theta_0; X_1, ..., X_n)}$$

- If $\Lambda_n < A$, decide that H0 is true and stop;
- If $\Lambda_n > B$, decide that H1 is true and stop;
- ... or we continue collecting observation


### Wald's SPRT Bound
$$A = \log \frac{β}{1 − α} \text{, } B = \log \frac{1 − β}{α}$$

We would want to control the two types of error:
1. $\alpha =$ Pr(Deciding for $H_1$ when $H_0$ is true)
2. $\beta =$ Pr(Deciding for $H_0$ when $H_1$ is true)

### Code Snipet

In [3]:
def sprt_proportion(x, p0, p1, alpha, beta, start=1, plot=False):
    if not (0 <= p0 <= 1 and 0 <= p1 <= 1):
        raise ValueError('Proportion must lie in [0, 1]')
    if not (0 < alpha < 1 and 0 < beta < 1):
        raise ValueError('Error control param must lie in (0, 1)')
    lr = start
    s_fac = math.log(p1 / p0)
    f_fac = math.log((1.0-p1) / (1.0-p0))
    s_bnd = math.log((1-beta) / alpha)
    f_bnd = math.log(beta / (1-alpha))
    decision = 'Unknown'
    for idx, trail in enumerate(x, 1):
        trail = int(trail)
        if trail != 0 and trail != 1:
            raise ValueError('every trail in the data must be a truthy/falsy value.')
        lr += s_fac if trail == 1 else f_fac
        if lr >= s_bnd:
            decision = 'Reject'
            break
        elif lr <= f_bnd:
            decision = 'Accept'
    return (decision, lr, idx)

# Kaplan-Wald
This function Calculates the confidence interval for the mean of a nonnegative random variable
       using the Kaplan-Wald method.

We want to find the lower confidence bound for the mean of a nonnegative random variable

- We have a sequence of nonnegative random variables, iid, $X_1, \ldots, X_n$
- Drawn from the same distribution, $F$
- Thier common expection is $\mu \equiv \mathbb E X_i = \int_0^\infty x dF$

### Code Snipet

In [4]:
def kaplan_wald_CI(x, level=0.99, lo=0, hi=float('inf'), gamma=0.95, xtol=1.e-32):
    alpha = 1.0 - level
    if not (0 < level < 1):
        raise ValueError('CI level must be between 0 and 1.')
    if any(x < 0):
        raise ValueError('Data x must be nonnegative.')
    if lo > hi or any(x < lo) or any(x > hi):
        raise ValueError('Data x is not in the specified range.')
    def find_t(data, start):
        f = lambda t: (np.max(np.cumsum(np.log(gamma*data/t + 1-gamma))) + np.log(alpha))
        start, end = start, np.mean(data)
        if f(start) * f(end) <= 0.0:
            return sp.optimize.brentq(f, start, end, xtol=xtol)
        else:
            return start
    lo = find_t(x, lo + xtol)
    if hi != float('inf'):
        hi = find_t(hi - x, xtol) + np.mean(x)
    return (lo, hi)

# Results, Next Steps have 

The functions we developed for the permutation test have been written and packaged in a pull request that requires further review. We will continue to improve on the functions as feedback is given from the Permute Package contributors until the final request is pulled. As of late, our unit test coverage for the package maintains the desired 98% production coverage.

Pull Request Link: https://github.com/statlab/permute/pull/134

# Bibliography

* The _permute_ developer guide: http://statlab.github.io/permute/dev/index.html
* Lehmann, E., _Nonparametrics: Statistical Methods Based on Ranks_, 1st ed. 1975; revised edition 2006
* Stark, P.B., _SticiGui_, http://statistics.berkeley.edu/users/stark/SticiGui, 1997--2017
* David Freedman and David Lane, _A Nonstochastic Interpretation of Reported Significance Levels_, ed. 1983

# Acknowledgements

**Andrew Linxie**: Wrote `one_sample_percentile`, `one_sample_percentile_ci`, `one_sample` (modified), and `one_sample_conf_int` functions in `permute.core` as well as unit tests for them.

**Michael Zheng**: Wrote `hoeffding_conf_int`, and `reg_coeff` as well as unit tests for them.

**Steven Wu**: Wrote non-permutation method `sprt_proportion`, `kaplan_wald_CI`, and `two_sample_fit` (currently only supports KS) in core.py, and their tests.

Special thanks to Professor Stark and Kellie Ottoboni for help on the Permute Package and Nonparametric theory.